In [ ]:
import functools

import numpy as np
import matplotlib.pyplot as plt

import shapely.geometry
import skimage.draw

import tensorflow as tf

import pydicom

import pymedphys
import pymedphys._dicom.structure as dcm_struct

In [ ]:
dcm_paths = pymedphys.zip_data_paths("dummy-ct-and-struct.zip")

In [ ]:
dcm_headers = []

for dcm_path in dcm_paths:
    dcm_headers.append(pydicom.read_file(str(dcm_path), force=True, stop_before_pixels=True))

In [ ]:
ct_image_paths = {
    header.SOPInstanceUID: path
    for header, path in zip(dcm_headers, dcm_paths)
    if header.SOPClassUID.name == "CT Image Storage"
}

In [ ]:
structure_sets = {
    header.SOPInstanceUID: header
    for header in dcm_headers
    if header.SOPClassUID.name == "RT Structure Set Storage"
}

In [ ]:
a_ct_path = list(ct_image_paths.items())[0][1]
dcm = pydicom.read_file(str(a_ct_path), force=True)

In [ ]:
dcm.pixel_array

In [ ]:
structure_sets

In [ ]:
dcm[-1].SOPClassUID.name == "CT Image Storage"

In [ ]:
dcm[-1].SOPClassUID.name

In [ ]:
dcm[0]

In [ ]:
dcm_path

In [ ]:
dcm_path

In [ ]:
dcm

In [ ]:
dcm_struct.list_structures(dcm)

In [ ]:
i = 7
x, y, z = dcm_struct.pull_structure('ANT Box', dcm)

x = x[i]
y = y[i]

In [ ]:
plt.plot(x, y)
plt.axis('equal')

In [ ]:
dx, dy = 2, 2
Cx, Cy = -100, -300

Ox, Oy = 1, 1

In [ ]:
r = (y - Cy) / dy * Oy
c = (x - Cx) / dx * Ox

In [ ]:
r

In [ ]:
np.array(list(zip(r*4, c*4)))

In [ ]:
img_size = 128
expansion = 4

In [ ]:
expanded_mask = skimage.draw.polygon2mask((img_size * expansion, img_size * expansion), np.array(list(zip(r*expansion, c*expansion))))

In [ ]:
plt.pcolormesh(expanded_mask)

In [ ]:
def reduce_expanded_mask(expanded_mask, img_size, expansion):
    expanded_mask = tf.dtypes.cast(expanded_mask, tf.float32)
    return tf.reduce_mean(
        tf.reduce_mean(
            tf.reshape(expanded_mask, (img_size, expansion, img_size, expansion)),
            axis=1,
        ),
        axis=2,
    )

In [ ]:
mask = reduce_expanded_mask(expanded_mask, img_size, expansion)

In [ ]:
plt.pcolormesh(mask)